In [6]:
#Import libraries
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as st

In [8]:
#Read CSV into PANDAS
cars = '../data/vehicle-fuel-type-count-by-zip-code.csv'
geocodes = '../data/fips_zip_correct.csv'
cars_data = pd.read_csv(cars)
geocodes = pd.read_csv(geocodes)

FileNotFoundError: [Errno 2] File ../data/vehicle-fuel-type-count-by-zip-code.csv does not exist: '../data/vehicle-fuel-type-count-by-zip-code.csv'

In [3]:
#Get number of unique values for counties
geocodes1 = geocodes[geocodes['State'] == 6]
geocodes1.nunique()

TypeError: string indices must be integers

In [9]:
#Get rid of duplicate cars
cars_data.nunique()

Date             1
Zip Code      2915
Model Year      15
Fuel             9
Make           113
Duty             2
Vehicles      2375
dtype: int64